**Time Series focasting using FB PROPHET LIBRARY**

In [ ]:
#installing the package
#!pip install fbprophet # this is the model package  to be used
#!pip install pystan
#!pip install yfinance # for loading the dataset to be used. data can be loaded normally thus no need to install this package.

In [ ]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
import fbprophet
%matplotlib inline
import yfinance as yf

In [ ]:
#read time series data from Yahoo Finance by yfinance
HSI = yf.Ticker("^HSI") 
start = datetime.datetime(2015,1,1)
end = datetime.datetime(2021,9,5)
hist = HSI.history(start=start, end=end, auto_adjust=True)
hist
#if import data via url: 
url="https://query1.finance.yahoo.com/v7/finance/download/^HSI?period1=1420066800&period2=1622437200&interval=1d&events=history&crumb=tO1hNZoUQeQ"  


In [ ]:
data=pd.read_csv(url)
data[:5]

In [ ]:
data.tail()

In [ ]:
#checking for null values
data.isna().sum()

In [ ]:
data["Close"].plot();

In [ ]:
data.dropna(inplace=True)
data.isnull().sum()

### For fbprophet to work, it require renaming of the column i.e ds for date and y for target variable

In [ ]:
#show the data table in Date, Stock Price (Close)
data_df= data[["Date", "Close"]]
data_df.head()

In [ ]:
#renaming columns ds and y as required by the fbprophet.
data_df.columns=["ds", "y"]
data_df.head()

In [ ]:
#changing date/ds to datetime
from datetime import datetime
data_df["ds"] = pd.to_datetime(data_df["ds"])


In [ ]:
data_df.info()

In [ ]:
#importing prophet

from fbprophet import Prophet


In [ ]:
#checking the function in the prophet package

dir(Prophet)

## Modeling

In [ ]:
#Initializing the model

model=Prophet()

In [ ]:
#fitting the model
model.fit(data_df)

In [ ]:
#checking on which seasonality the model was build on
model.component_modes

In [ ]:
#creating future 365 dates to focast

future_dates= model.make_future_dataframe(periods=365)

#checking 
future_dates.tail()



In [ ]:
#making predictions on future dates
pred= model.predict(future_dates)
pred.tail()

In [ ]:
#plotting the prediction
visualize_model=model.plot(pred)## assign the model to avoid plotting the visual in double

Observation: 
* The dotted line are the real data.
* Dark Blue line, are the projections
* light blue are the trend space.

* The point without real data are the future prediction for the next one year.

In [ ]:
#we can predict confidence interval only
pred.columns

In [ ]:
#specifying the column needed
focasting= pred[['ds', 'yhat_lower','yhat_upper', 'yhat' ]]
visualize_model=model.plot(focasting);

In [ ]:
#checking trend and weekly. monthly and yearly prediction

model.plot_components(pred);

### Cross Validation

in time series, the data is sequential and thus cross validation is done differently.

In above case where our data is frm 2015 to 2021, we can take;
 2015-2020 data as our training dataset
 2020-2021 data as our test data
 2021-2022 is  validation dataset.. completely new dataset used for prediction.
 

In [ ]:
#show all the forecasting values in a table
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pred[["ds", "yhat", "yhat_lower", "yhat_upper"]][:10]

In [ ]:
#we can use fpprophet cross validation inbuild function

from fbprophet.diagnostics import cross_validation

data_cv = cross_validation(model, initial="730 days", period="180 days", horizon="365 days")
data_cv.head()

In [ ]:
#cheking the tail
data_cv.tail()

Observation: 
* initial days= today number of days we want (2*horizon days)
* period= half of the the horizon days (horizon/2)# u can give a better nearest number.
* horizon= total number of days to be predicted


the output is ds, yhat, yhat_lower, yhat_upper, y and cutoff.


In [ ]:
#show all the forecasting and predicted forecasted values in a table
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
data_cv[["ds", "y", "yhat"]]. head(20)

In [ ]:
#checking perfomance metrics
from fbprophet.diagnostics import performance_metrics

data_p = performance_metrics(data_cv)
data_p.head()

observation: perfomance metrics is computed on y column and yhat columns

In [ ]:
#we can plot the above metrics
from fbprophet.plot import plot_cross_validation_metric
fig= plot_cross_validation_metric(data_cv, metric='mse');

**observation**: this is the preditions of the future dates.